In [1]:
import torch.nn as nn
import torch
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [7]:
class cnn(nn.Module):
    def __init__(self):
        super(cnn, self).__init__()
        self.conv1 = nn.Conv2d(3,64,3)
    def forward(self, x):
        return self.conv1(x)
    
model = cnn()

# Transform 전처리기 생성

In [22]:
import torchvision.transforms as T
transforms = T.Compose([T.ToTensor()])

# 데이터셋 세팅

In [111]:
from torch.utils.data import Dataset
from PIL import Image

class HmDataset(Dataset):
    def __init__(self, df_path, transforms=None):
        self.df = pd.read_csv(df_path)
        self.transforms = transforms
        
    def __getitem__(self, index):
        hm_meta = self.df.iloc[index]
        filename = hm_meta.filename
        label = hm_meta['epidural':'any'].values.astype(np.float)
        
        img = Image.open('../dataset/kaggle_rsna(only100)/imgs/'+filename+'.png')
        if self.transforms is not None:
            img = self.transforms(img)
        
        return filename, label, img
    
    def __len__(self):
        return len(self.df)

train_dataset = HmDataset(df_path='./train.csv', transforms=transforms)
valid_dataset = HmDataset(df_path='./valid.csv', transforms=transforms)
test_dataset = HmDataset(df_path='./valid.csv', transforms=transforms)

In [112]:
from torch.utils.data import DataLoader
# Hyper Parameter
BATCH_SIZE = 16

train_loader = DataLoader(train_dataset,
                         batch_size=BATCH_SIZE,
                         shuffle=True,
                         num_workers=4)
valid_loader = DataLoader(valid_dataset,
                         batch_size=BATCH_SIZE,
                         shuffle=False,
                         num_workers=4)

# 학습